# Setup

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import optimize
import statistics as stat
from IPython.display import display
import math

class par: None
class moms: None

## Settings and assumptions

In [ ]:
#Shock size (used in step 3)
par.b = par.bh = 0.15
#Risk aversion coefficient (used in step 3)
par.theta = 12
#IES, sigma = 1/IES (used in step 3)
par.sigma = 0.5

# Load data

In [ ]:
df = pd.read_csv("data/V3_timeseries__postrev_I1Y_19-Oct-2018.csv")
df = df[['year','ProfitK','ProfitY','RF', 'growthpop','priceinvt','PD','growthTFP','ik','EmpPop']]
df = df.set_index('year')
#display(df.head())
#display(df.describe())

# Moments

In [ ]:
# Start and end dates defined in main notebook
moms.ProfitK = stat.mean(df.loc[start:end,"ProfitK"])/100
moms.s_K = stat.mean(df.loc[start:end,"ProfitY"])/100
moms.ik = stat.mean(df.loc[start:end,"ik"])/100
moms.PD = stat.mean(df.loc[start:end,"PD"])
moms.TFPgrowth = stat.mean(df.loc[start:end,"growthTFP"])/100
moms.RF = stat.mean(df.loc[start:end,"RF"])/100 #used in step 3

#for name in ['ProfitK','s_K','ik','PD','TFPgrowth','RF']:
#    print(f'{name:10s} = {getattr(moms,name):.5f}')

# Step 1

Estimate $g_L$, $g_Q$ and $\bar{N}$ directly from data

In [ ]:
par.g_L = stat.mean(df.loc[start:end,"growthpop"])/100
par.g_Q = -stat.mean(df.loc[start:end,"priceinvt"])/100 # note: negativ value used
par.N_bar = stat.mean(df.loc[start:end,"EmpPop"])/100

#for name in ['g_L','g_Q','N_bar']:
#    print(f'{name:5s} = {getattr(par,name):.5f}')

# Step 2

Estimate $\mu$, $\alpha$, $\delta$, $g_z$ by solving equation system

In [ ]:
def g_T(par):
    par.g_T = (1+par.g_L)*((1+par.g_Z)**(1/(1-par.alpha)))*((1+par.g_Q)**(par.alpha/(1-par.alpha)))-1
   
def eq_footnote_15(par,moms):
     return moms.TFPgrowth -(par.g_T-(1-moms.s_K)*par.g_L-moms.s_K*(par.g_T+par.g_Q))

def eq_11(par,moms):
    return (1+par.g_L)*((1+par.g_Z)**(1/(1-par.alpha)))*((1+par.g_Q)**(par.alpha/(1-par.alpha)))-(1+par.g_T)

def eq_18(par,moms):
    return (1+par.g_Q)*(1+par.g_T)-(1-par.delta) - moms.ik

def eq_20(par,moms):
    return (par.mu+par.alpha-1)/(par.mu)-moms.s_K

In [ ]:
def set_parameters(par,x):
    
    for name,value in zip(par.names,x):
        setattr(par,name,value)
    
    # update g_T
    g_T(par)
    
def set_x(par):
    
    x = np.zeros(len(par.names))
    for i,name in enumerate(par.names):
        x[i] = getattr(par,name)
        
    return x

def eq_system(x,par,moms):
    
    # a. set parameters
    set_parameters(par,x)            
    
    # c. evaluate equations
    out = []
    out.append(eq_footnote_15(par,moms))
    out.append(eq_11(par,moms))
    out.append(eq_18(par,moms))
    out.append(eq_20(par,moms))

    return out

In [ ]:
# a. parameters to estimate
par.names = ['mu','delta','alpha','g_Z']

# b. guess
par.mu = 1.078
par.delta = 0.02778
par.alpha = 0.244
par.g_Z = 0.01298
x = set_x(par)

# c. solve
solution = optimize.fsolve(eq_system, x, args=(par,moms), full_output=0)
set_parameters(par,solution)

# d. print result
#for name in par.names:
#    print(f'{name:10} = {getattr(par,name):4f}')

# Step 3

Estimate $\beta$ and $p$

In [ ]:
# Calculate r_star
par.r_star = ((1 + par.g_T ) / moms.PD ) + par.g_T
# Calculate g_PC
par.g_PC = (1+par.g_T)/(1 + par.g_L) - 1

In [ ]:
# define e^(r*-rf)
exp_spread = math.exp(par.r_star - moms.RF)
# Find analytical derivation of p in appendix
par.p = (exp_spread - 1) / (((1 - par.b)**(-par.theta) + (1 + par.bh)**(-par.theta) -2) - exp_spread * ((1 - par.b)**(1-par.theta) + (1 + par.bh)**(1-par.theta) -2))
#Estimate rho and thus beta
par.rho = par.r_star - (par.sigma * par.g_PC + par.sigma * ((1-(1/par.sigma))/(1-par.theta)) * math.log(1 + par.p * ((1 - par.b)**(1-par.theta) + (1 + par.bh)**(1-par.theta) - 2)))
par.beta = 1/(1+par.rho)

In [ ]:
#Print all results
#for name in ['beta','mu','p','delta','alpha','g_L', 'g_Z', 'g_Q', 'N_bar']:
#    print(f'{name:10} = {getattr(par,name):4f}')

# Store values

In [ ]:
df_estimates = []
for name in ['beta','mu','p','delta','alpha','g_L', 'g_Z', 'g_Q', 'N_bar']:
    df_estimates.append({'Name': name, str(start) + ' - ' + str(end):getattr(par,name)})

df_estimates = pd.DataFrame(df_estimates)
df_estimates = df_estimates.set_index('Name')

In [ ]:
# Magic - stores across notebooks
%store df_estimates